In [1]:
import music21
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import mnet
from scipy import stats

## Load Data

In [2]:
'''
Creat local corpus with access pieces
'''

localCorpus = music21.corpus.corpora.LocalCorpus()
localCorpus.addPath('../library')
music21.corpus.cacheMetadata()


/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: local metadata cache: starting processing of paths: 1
/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
bundles.py: WARNING: MetadataBundle Modification Time: 1589091893.0191493
bundles.py: WARNING: Skipped 1 sources already in cache.
/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: writing time: 0.025 md items: 6

/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
caching.py: WARNING: cache: final writing time: 0.025 seconds


## Generate Community-based rhythm

## Roman Numeral 

In [5]:

s = music21.corpus.parse('telemannfantasie1.xml')
chord_lst = list(s.chordify().recurse().notes)
nodelst = mnet.convert_chord_note(chord_lst, 'A')
g_rn=mnet.create_graph(nodelst)

#write as .gexf
#nx.write_gexf(g_rn, "rn_composition_notechord.gexf")


## Clustering

Infomap analysis done in web app

In [146]:

g_info = mnet.convert_to_weighted(g_rn, fraction=False)

#Node to community dictionary
bigDict = {}
smallDict = {}
completeDict = {}
with open("../graphs/rn.net.tree") as tree:
    for line in tree:
        spl = line.split('"')

        node = spl[1]

        
        path = spl[0].split(':')
        bigGroup=path[0]
        smallGroup=path[0]+path[1]
        
        #make complete group
        completeGroup=spl[0].split(" ")[0]
        completeGroup = ':'.join(completeGroup.split(':')[0:-1])
        print(completeGroup)
        #print(node, bigGroup, smallGroup)
        bigDict[node]=bigGroup
        smallDict[node]=smallGroup
        completeDict[node]=completeGroup
        
        
nx.set_node_attributes(g_info, bigDict, 'bigGroup')
nx.set_node_attributes(g_info, smallDict, 'smallGroup')
#nx.write_gexf(g_info, "../graphs/rn_info.gexf")


#community to Node dictionaries + list of subgraphs
c1, sglst1= mnet.generateCommunities(bigDict, g_info)

c2, sglst2 = mnet.generateCommunities(smallDict, g_info)

1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:1
1:2
1:2
1:2
1:2
1:2
1:2
1:2
1:2
1:3
1:3
1:3
1:3
1:3
1:3
1:3
1:3
1:3
1:4
1:4
1:4
1:4
1:4
1:4
1:4
1:5
1:5
1:5
1:5
1:5
1:5
1:5
1:5
1:6
1:6
1:6
1:6
1:6
1:7
1:7
1:7
1:7
1:7
1:7
1:8
1:8
1:8
1:8
1:8
1:8
1:9
1:9
1:9
1:9
1:9
1:9
1:9
1:10
1:10
1:10
1:10
1:10
1:10
1:11
1:11
1:11
1:11
1:11
1:12:1
1:12:1
1:12:1
1:12:1
1:12:1
1:12:2
1:12:2
1:13
1:13
1:13
1:13
1:13
1:13
1:14
1:14
1:14
1:14
1:14
1:15
1:15
1:15
1:15
1:15
1:16
1:16
1:16
1:16
1:17
1:17
1:17
1:17
1:18
1:18
1:18
2:1
2:1
2:1
2:1
2:1
2:1
2:2
2:2
2:2
2:2
2:2
2:3
2:3
2:3
2:3
2:3
2:3
2:4
2:4
2:4
2:4
2:4
3:1:1
3:1:1
3:1:1
3:1:2
3:1:2
3:1:2
3:1:2
3:1:2
3:1:3
3:1:3
3:1:3
3:1:4
3:1:4
3:1:4
3:2:1
3:2:1
3:2:1
3:2:1
3:2:1
3:2:2
3:2:2
3:2:2
3:2:2
3:2:2
3:2:3
3:2:3


## Original Annotated

In [165]:
'''
Assigns note lengths based on community strucuture. Every time a community switches, note will be lengthend accoridng to 
noteLengthAssign. 


noteLengthAssign: list with lengths of note types. Default length, then ascending for community changes
levels: layers of hirarchical structure
nodeToGroupDict: Dictionary with community assignment of each node. Assignments must be in the from a:b:c, where
                b is embedded in a, and c is embedded in b
Long position: where longer note should be places (end of current community or beginning of next community)

'''
def str_commmunity_rhythm(randomwalk, noteLengthAssign, nodeToGroupDict, long_position = 'cur_comm'):
        
        
    #randomwalk.append('pad long boo')
    mellst = []
    lyriclst = []
    i = 0
    while i < len(randomwalk)-1:
        #print(i)
        node = randomwalk[i]
        mel = node.split()[0]
        n = music21.note.Note(mel)
        community= nodeToGroupDict[node]
        n.lyric = community
        lyriclst.append(community)
        
        #determine community equality
        community_next = nodeToGroupDict[randomwalk[i+1]]
        
        com_lst = community.split(":")

        com_next_lst = community_next.split(":")
        
        if com_lst == com_next_lst:

            n.duration.quarterLength = noteLengthAssign[0]
            #print(noteLengthAssign[0])
            mellst.append(n)
        else:
            

            p=0
            pmax = min([len(com_lst), len(com_next_lst)])
            while p < pmax:
                print("p is ", p)
                print(com_lst)
                print(com_next_lst)
                if com_lst[p] != com_next_lst[p]: #if group not equal
                    emph_note_len = noteLengthAssign[p+1] 
                    break
                p += 1

            #next comm condition
            if long_position == 'next_comm':
                
                n.duration.quarterLength = noteLengthAssign[0]
                mellst.append(n)
                
                i += 1 # go to next note
                #print(i)
                nextNote = music21.note.Note(randomwalk[i].split()[0])
                nextNote.duration.quarterLength = emph_note_len
                print("longer note: ", emph_note_len)
                nextNote.lyric=community_next
                mellst.append(nextNote)

                    
            elif long_position == 'cur_comm':
               
                n.duration.quarterLength = emph_note_len
            
                mellst.append(n)
                    
                

        i +=1
        
        
  
    return mellst, lyriclst
    
    
    
    

In [155]:
randomwalk=mnet.generate_randomwalk(g_rn)



In [158]:
noteLengthAssign = [.5, 2, 1, .25]
melody_list_small, llst= str_commmunity_rhythm(randomwalk, noteLengthAssign, completeDict)


p is  0
['1', '1']
['1', '3']
p is  1
['1', '1']
['1', '3']
longer note:  1
p is  0
['1', '3']
['1', '7']
p is  1
['1', '3']
['1', '7']
longer note:  1
p is  0
['1', '7']
['1', '3']
p is  1
['1', '7']
['1', '3']
longer note:  1
p is  0
['1', '3']
['1', '2']
p is  1
['1', '3']
['1', '2']
longer note:  1
p is  0
['1', '2']
['1', '1']
p is  1
['1', '2']
['1', '1']
longer note:  1
p is  0
['1', '1']
['2', '1']
longer note:  2
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
longer note:  1
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
longer note:  1
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
longer note:  1
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
longer note:  1
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
longer note:  1
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
longer note:  1
p is  0
['2', '1']
['2', '2']
p is  1
['2', '1']
['2', '2']
longer note:  1
p is  0
['2', '2']
['1', '5']
longer note:

In [159]:
melody_stream_small = mnet.convert_to_stream(melody_list_small)
melody_stream_small.write("xml", "../xml/RN_rhythm_aftergroup.xml")

'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/RN_rhythm_aftergroup.xml'

# Strengthen Edges Between Large Communities

In [9]:
bridge12, bridge21 = mnet.findBridges(g_info, c1['1'], c1['2'])

In [10]:
bridge13, bridge31 = mnet.findBridges(g_info, c1['1'], c1['3'])

In [11]:
g_rn1 = g_rn.copy()
g_rn1 = mnet.degree_increase(g_rn1, bridge13, 20)
g_rn1 = mnet.degree_increase(g_rn1, bridge12, 20)

In [166]:
randomwalk1 =mnet.generate_randomwalk(g_rn1)


In [167]:
noteLengthAssign = [.5, 2, 1, .25]
melody_list1, llst= str_commmunity_rhythm(randomwalk1, noteLengthAssign, completeDict)
melody_stream1 = mnet.convert_to_stream(melody_list1)

p is  0
['1', '1']
['1', '6']
p is  1
['1', '1']
['1', '6']
p is  0
['1', '6']
['1', '1']
p is  1
['1', '6']
['1', '1']
p is  0
['1', '1']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '2', '1']
p is  1
['3', '2', '3']
['3', '2', '1']
p is  2
['3', '2', '3']
['3', '2', '1']
p is  0
['3', '2', '1']
['3', '2', '2']
p is  1
['3', '2', '1']
['3', '2', '2']
p is  2
['3', '2', '1']
['3', '2', '2']
p is  0
['3', '2', '2']
['3', '2', '3']
p is  1
['3', '2', '2']
['3', '2', '3']
p is  2
['3', '2', '2']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '2', '1']
p is  1
['3', '2', '3']
['3', '2', '1']
p is  2
['3', '2', '3']
['3', '2', '1']
p is  0
['3', '2', '1']
['3', '2', '2']
p is  1
['3', '2', '1']
['3', '2', '2']
p is  2
['3', '2', '1']
['3', '2', '2']
p is  0
['3', '2', '2']
['3', '2', '3']
p is  1
['3', '2', '2']
['3', '2', '3']
p is  2
['3', '2', '2']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '1', '4']
p is  1
['3', '2', '3']
['3', '1', '4']
p is  0
['3', '1', '4']
['3', '1', '1']
p is 

p is  0
['1', '8']
['1', '9']
p is  1
['1', '8']
['1', '9']
p is  0
['1', '9']
['1', '3']
p is  1
['1', '9']
['1', '3']
p is  0
['1', '3']
['1', '7']
p is  1
['1', '3']
['1', '7']
p is  0
['1', '7']
['1', '3']
p is  1
['1', '7']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '2', '1']
p is  1
['3', '2', '3']
['3', '2', '1']
p is  2
['3', '2', '3']
['3', '2', '1']
p is  0
['3', '2', '1']
['3', '2', '2']
p is  1
['3', '2', '1']
['3', '2', '2']
p is  2
['3', '2', '1']
['3', '2', '2']
p is  0
['3', '2', '2']
['3', '2', '3']
p is  1
['3', '2', '2']
['3', '2', '3']
p is  2
['3', '2', '2']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '2', '1']
p is  1
['3', '2', '3']
['3', '2', '1']
p is  2
['3', '2', '3']
['3', '2', '1']
p is  0
['3', '2', '1']
['3', '2', '2']
p is  1
['3', '2', '1']
['3', '2', '2']
p is  2
['3', '2', '1']
['3', '2', '2']
p is  0
['3', '2', '2']
['3', '2', '3']
p is  1
['3', '2', '2']
[

p is  1
['1', '4']
['1', '3']
p is  0
['1', '3']
['1', '16']
p is  1
['1', '3']
['1', '16']
p is  0
['1', '16']
['1', '3']
p is  1
['1', '16']
['1', '3']
p is  0
['1', '3']
['1', '7']
p is  1
['1', '3']
['1', '7']
p is  0
['1', '7']
['1', '3']
p is  1
['1', '7']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['1', '3']
p is  1
['1', '1']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '2', '1']
p is  1
['3', '2', '3']
['3', '2', '1']
p is  2
['3', '2', '3']
['3', '2', '1']
p is  0
['3', '2', '1']
['3', '2', '2']
p is  1
['3', '2', '1']
['3', '2', '2']
p is  2
['3', '2', '1']
['3', '2', '2']
p is  0
['3', '2', '2']
['3', '2', '3']
p is  1
['3', '2', '2']
['3', '2', '3']
p is  2
['3', '2', '2']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '1', '4']
p is  1
['3', '2', '3']
['3', '1', '4']
p is  0
['3', '1', '4']
['3', '1', '1']
p is  1
['3', '1', '4']
['3', '

In [168]:
melody_stream1.write("xml", "../xml/IncreaseTrafficAwayFromComm1.xml")

'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/IncreaseTrafficAwayFromComm1.xml'

## Strengthen Sequences

In [169]:
g_rn2 = g_rn.copy()

Sequences usually have 1 less edge than nodes

In [170]:
for subgraph in sglst2:
    if len(list(subgraph.nodes())) - 1 == len(list(subgraph.edges())):
        for node in list(subgraph.nodes):
            if len(list(subgraph.predecessors(node))) == 0: #entrance node
                preds = g_rn2.predecessors(node)
                prededgelst=[]
                for p in preds:
                    prededgelst.append((p, node))
                g_rn2 = mnet.degree_increase(g_rn2, prededgelst, 10)
                    
                
        

In [173]:
randomwalk2 =mnet.generate_randomwalk(g_rn2)

In [177]:
noteLengthAssign = [.5, 2, 1, .25]
melody_list2, llst= str_commmunity_rhythm(randomwalk2, noteLengthAssign, completeDict)

p is  0
['1', '1']
['1', '18']
p is  1
['1', '1']
['1', '18']
p is  0
['1', '18']
['1', '8']
p is  1
['1', '18']
['1', '8']
p is  0
['1', '8']
['1', '15']
p is  1
['1', '8']
['1', '15']
p is  0
['1', '15']
['1', '7']
p is  1
['1', '15']
['1', '7']
p is  0
['1', '7']
['1', '17']
p is  1
['1', '7']
['1', '17']
p is  0
['1', '17']
['1', '1']
p is  1
['1', '17']
['1', '1']
p is  0
['1', '1']
['1', '18']
p is  1
['1', '1']
['1', '18']
p is  0
['1', '18']
['1', '8']
p is  1
['1', '18']
['1', '8']
p is  0
['1', '8']
['1', '15']
p is  1
['1', '8']
['1', '15']
p is  0
['1', '15']
['1', '7']
p is  1
['1', '15']
['1', '7']
p is  0
['1', '7']
['1', '17']
p is  1
['1', '7']
['1', '17']
p is  0
['1', '17']
['1', '1']
p is  1
['1', '17']
['1', '1']
p is  0
['1', '1']
['1', '14']
p is  1
['1', '1']
['1', '14']
p is  0
['1', '14']
['1', '8']
p is  1
['1', '14']
['1', '8']
p is  0
['1', '8']
['1', '15']
p is  1
['1', '8']
['1', '15']
p is  0
['1', '15']
['1', '7']
p is  1
['1', '15']
['1', '7']
p is  0


['1', '1']
['1', '8']
p is  0
['1', '8']
['1', '9']
p is  1
['1', '8']
['1', '9']
p is  0
['1', '9']
['1', '3']
p is  1
['1', '9']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['1', '18']
p is  1
['1', '1']
['1', '18']
p is  0
['1', '18']
['1', '8']
p is  1
['1', '18']
['1', '8']
p is  0
['1', '8']
['1', '15']
p is  1
['1', '8']
['1', '15']
p is  0
['1', '15']
['1', '7']
p is  1
['1', '15']
['1', '7']
p is  0
['1', '7']
['1', '17']
p is  1
['1', '7']
['1', '17']
p is  0
['1', '17']
['1', '1']
p is  1
['1', '17']
['1', '1']
p is  0
['1', '1']
['1', '3']
p is  1
['1', '1']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['1', '18']
p is  1
['1', '1']
['1', '18']
p is  0
['1', '18']
['1', '8']
p is  1
['1', '18']
['1', '8']
p is  0
['1', '8']
['1', '15']
p is  1
['1', '8']
['1', '15']
p is  0
['1', '15']
['1', '7']
p is  1
['1', '15']
['1', '7']
p is  0
['1', '7']
['1', '17']
p is  1
['1', '7']
['1', '1

p is  0
['1', '1']
['1', '3']
p is  1
['1', '1']
['1', '3']
p is  0
['1', '3']
['1', '16']
p is  1
['1', '3']
['1', '16']
p is  0
['1', '16']
['1', '3']
p is  1
['1', '16']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['1', '10']
p is  1
['1', '1']
['1', '10']
p is  0
['1', '10']
['2', '1']
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
p i

['2', '3']
p is  1
['2', '1']
['2', '3']
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
p is  0
['2', '1']
['2', '3']
p is  1
['2', '1']
['2', '3']
p is  0
['2', '3']
['2', '4']
p is  1
['2', '3']
['2', '4']
p is  0
['2', '4']
['2', '1']
p is  1
['2', '4']
['2', '1']
p is  0
['2', '1']
['1', '4']
p is  0
['1', '4']
['1', '3']
p is  1
['1', '4']
['1', '3']
p is  0
['1', '3']
['1', '7']
p is  1
['1', '3']
['1', '7']
p is  0
['1', '7']
['1', '3']
p is  1
['1', '7']
['1', '3']
p is  0
['1', '3']
['1', '16']
p is  1
['1', '3']
['1', '16']
p is  0
['1', '16']
['1', '3']
p is  1
['1', '16']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['1', '14']
p is  1
['1', '1']
['1', '14']
p is  0
['1', '14']
['1

In [184]:
melody_stream2 = mnet.convert_to_stream(melody_list2)
melody_stream2.write("xml", "../xml/RN_Strengthen_Sequences.xml")

'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/RN_Strengthen_Sequences.xml'

## Strengthen Non - Sequences

In [179]:
g_rn3 = g_rn.copy()

In [180]:
for subgraph in sglst2:
    if len(list(subgraph.nodes())) - 1 != len(list(subgraph.edges())):
        #print("found")
        for node in list(subgraph.nodes):

            if len(list(subgraph.predecessors(node))) != len(list(g_rn3.predecessors(node))): #entrance node
                preds = set(g_rn3.predecessors(node)) - set(subgraph.predecessors(node))
                prededgelst=[]
                for p in preds:
                    prededgelst.append((p, node))
                g_rn3 = mnet.degree_increase(g_rn3, prededgelst, 10)
                #print("edges added")
                #print(prededgelst)
                    
                
        

In [181]:
randomwalk3 =mnet.generate_randomwalk(g_rn3)

In [182]:
noteLengthAssign = [.5, 2, 1, .25]
melody_list3, llst= str_commmunity_rhythm(randomwalk3, noteLengthAssign, completeDict)

p is  0
['1', '1']
['1', '6']
p is  1
['1', '1']
['1', '6']
p is  0
['1', '6']
['1', '1']
p is  1
['1', '6']
['1', '1']
p is  0
['1', '1']
['1', '6']
p is  1
['1', '1']
['1', '6']
p is  0
['1', '6']
['1', '1']
p is  1
['1', '6']
['1', '1']
p is  0
['1', '1']
['1', '8']
p is  1
['1', '1']
['1', '8']
p is  0
['1', '8']
['1', '2']
p is  1
['1', '8']
['1', '2']
p is  0
['1', '2']
['1', '1']
p is  1
['1', '2']
['1', '1']
p is  0
['1', '1']
['1', '13']
p is  1
['1', '1']
['1', '13']
p is  0
['1', '13']
['1', '2']
p is  1
['1', '13']
['1', '2']
p is  0
['1', '2']
['1', '1']
p is  1
['1', '2']
['1', '1']
p is  0
['1', '1']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '1', '4']
p is  1
['3', '2', '3']
['3', '1', '4']
p is  0
['3', '1', '4']
['3', '1', '1']
p is  1
['3', '1', '4']
['3', '1', '1']
p is  2
['3', '1', '4']
['3', '1', '1']
p is  0
['3', '1', '1']
['3', '1', '2']
p is  1
['3', '1', '1']
['3', '1', '2']
p is  2
['3', '1', '1']
['3', '1', '2']
p is  0
['3', '1', '2']
['3', '1', '3']
p

p is  0
['3', '2', '3']
['3', '1', '4']
p is  1
['3', '2', '3']
['3', '1', '4']
p is  0
['3', '1', '4']
['3', '1', '1']
p is  1
['3', '1', '4']
['3', '1', '1']
p is  2
['3', '1', '4']
['3', '1', '1']
p is  0
['3', '1', '1']
['3', '1', '2']
p is  1
['3', '1', '1']
['3', '1', '2']
p is  2
['3', '1', '1']
['3', '1', '2']
p is  0
['3', '1', '2']
['3', '1', '3']
p is  1
['3', '1', '2']
['3', '1', '3']
p is  2
['3', '1', '2']
['3', '1', '3']
p is  0
['3', '1', '3']
['1', '1']
p is  0
['1', '1']
['1', '8']
p is  1
['1', '1']
['1', '8']
p is  0
['1', '8']
['1', '9']
p is  1
['1', '8']
['1', '9']
p is  0
['1', '9']
['1', '5']
p is  1
['1', '9']
['1', '5']
p is  0
['1', '5']
['1', '1']
p is  1
['1', '5']
['1', '1']
p is  0
['1', '1']
['1', '6']
p is  1
['1', '1']
['1', '6']
p is  0
['1', '6']
['1', '1']
p is  1
['1', '6']
['1', '1']
p is  0
['1', '1']
['3', '2', '3']
p is  0
['3', '2', '3']
['3', '1', '4']
p is  1
['3', '2', '3']
['3', '1', '4']
p is  0
['3', '1', '4']
['3', '1', '1']
p is  1
['

In [185]:
melody_stream3 = mnet.convert_to_stream(melody_list3)
melody_stream3.write("xml", "../xml/RN_Strengthen_NonSequences.xml")

'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/RN_Strengthen_NonSequences.xml'

## Increase Probability of Staying in Level 2 Community

In [186]:
g_rn5 = g_rn.copy()

In [187]:
for subgraph in sglst2:
    g_rn5 = mnet.degree_increase(g_rn5, list(subgraph.edges()), 10)


In [189]:
g_rn5 = g_rn.copy()


In [198]:
randomwalk5 =mnet.generate_randomwalk(g_rn5)

In [199]:
noteLengthAssign = [.5, 2, 1, .25]
melody_list5, llst= str_commmunity_rhythm(randomwalk5, noteLengthAssign, completeDict)


p is  0
['1', '1']
['1', '6']
p is  1
['1', '1']
['1', '6']
p is  0
['1', '6']
['1', '1']
p is  1
['1', '6']
['1', '1']
p is  0
['1', '1']
['1', '13']
p is  1
['1', '1']
['1', '13']
p is  0
['1', '13']
['1', '1']
p is  1
['1', '13']
['1', '1']
p is  0
['1', '1']
['1', '2']
p is  1
['1', '1']
['1', '2']
p is  0
['1', '2']
['1', '9']
p is  1
['1', '2']
['1', '9']
p is  0
['1', '9']
['1', '5']
p is  1
['1', '9']
['1', '5']
p is  0
['1', '5']
['1', '1']
p is  1
['1', '5']
['1', '1']
p is  0
['1', '1']
['1', '2']
p is  1
['1', '1']
['1', '2']
p is  0
['1', '2']
['1', '9']
p is  1
['1', '2']
['1', '9']
p is  0
['1', '9']
['1', '3']
p is  1
['1', '9']
['1', '3']
p is  0
['1', '3']
['1', '7']
p is  1
['1', '3']
['1', '7']
p is  0
['1', '7']
['1', '3']
p is  1
['1', '7']
['1', '3']
p is  0
['1', '3']
['1', '1']
p is  1
['1', '3']
['1', '1']
p is  0
['1', '1']
['1', '2']
p is  1
['1', '1']
['1', '2']
p is  0
['1', '2']
['1', '1']
p is  1
['1', '2']
['1', '1']
p is  0
['1', '1']
['1', '6']
p is  

In [200]:
melody_stream5 = mnet.convert_to_stream(melody_list5)
melody_stream5.write("xml", "../xml/RN_StayinL2.xml")


'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/RN_StayinL2.xml'